In [1]:
import librosa, matplotlib.pyplot as plt, numpy as np
from scipy.spatial.distance import mahalanobis
from sklearn.metrics import classification_report

# define functions
def mfcc_avg(files):
    avg = []
    for file in files:
        samples, samplingrate = librosa.load(file, sr=16000, mono=True, offset=0.0, duration=None)
        mfcc = librosa.feature.mfcc(y=samples, sr=samplingrate, win_length=400, hop_length=200, n_mfcc=13)
        avg.append(np.average(mfcc, axis=1))
    return np.average(avg, axis=0)

def distance(avg, test, cov_inv):
    return mahalanobis(avg, test, cov_inv)

def predict_speaker(test, avg_i, cov_inv):
    samples, samplingrate = librosa.load(test, sr=16000, mono=True, offset=0.0, duration=None)
    mfcc_test = librosa.feature.mfcc(y=samples, sr=samplingrate, win_length=400, hop_length=200, n_mfcc=13)
    avg_test = np.average(mfcc_test, axis=1)
    # Calculate and compare Mahalanobis distances
    d={}
    for ind in dataset:
        d[ind]= distance(avg_i[ind], avg_test, cov_inv)
    matchv=min(d, key= lambda k: d.get(k))
    return matchv

In [2]:
# Calculate average of MFCC features
# Dataset
dataset={
         'San':["vsa1.wav", "vsa2.wav", "vsa3.wav"],
         'Rose':["vro1.wav", "vro2.wav", "vro3.wav"],
         'Adi':["vak1.wav", "vak2.wav", "vak3.wav"],
         'Niru':["vni1.wav","vni2.wav","vni3.wav"],
         'Jay':["vja1.wav","vja2.wav","vja3.wav"],
         }

# Testset
testset = {
    'chkni.wav': 'Niru',
    'chkro.wav': 'Rose',
    'chksa.wav': 'San',
    'chkja.wav': 'Jay',
    'chkak.wav': 'Adi'
}

avg_i = {}
for ind in dataset:
    avg_i[ind] = mfcc_avg(dataset[ind])

# Calculate covariance matrix and its inverse for Mahalanobis distance
stack=np.vstack(list(avg_i.values()))
covm= np.cov(stack.T)
cov_inv = np.linalg.pinv(covm)

# Calculate metrics
true_labels = []
test_labels = []

for test, speaker in testset.items():
    true_labels.append(speaker)
    test_labels.append(predict_speaker(test, avg_i, cov_inv))

print(classification_report(true_labels,test_labels, zero_division=0))



              precision    recall  f1-score   support

         Adi       0.00      0.00      0.00         1
         Jay       1.00      1.00      1.00         1
        Niru       0.50      1.00      0.67         1
        Rose       1.00      1.00      1.00         1
         San       1.00      1.00      1.00         1

    accuracy                           0.80         5
   macro avg       0.70      0.80      0.73         5
weighted avg       0.70      0.80      0.73         5

